# How to navigate the ecosystem?

The Julia ecosystem comprises **over 5000 packages** (as of February 2021).

Exploring available packages: https://juliapackages.com and https://juliaobserver.com

Searching packages / their documentation / **their code**: https://juliahub.com/

GitHub organizations: https://julialang.org/community/organizations/

# Personal selection of packages
**(in no particular order)**

Let's walk over a couple of packages that you might find useful in the context of scientific numerical computing.

In [ ]:
] status

In [ ]:
using Pkg
pkg"activate ."
pkg"instantiate"
pkg"precompile"

### [Parameters.jl](https://github.com/mauro3/Parameters.jl)

In [ ]:
using Parameters

In [ ]:
@with_kw struct MyParams
    x::Int = 4
    y::Float64
end

In [ ]:
MyParams()

In [ ]:
MyParams(y = 2)

In [ ]:
@with_kw struct MyParams2
    x::Int = 4;
    y::Float64 = 5.3
    z::Int = floor(x+y)
end

In [ ]:
MyParams2()

In [ ]:
@with_kw struct MyParams3
    x::Int = 4; @assert x != 2
    y::Float64 = 5.3
    @assert x + y >= 9
end

In [ ]:
MyParams3(y = 3)

In [ ]:
MyParams3(x = 2)

In [ ]:
@with_kw mutable struct MyParams4 @deftype Int
    x = 4;
    y::Float64 = 5.3
    z = floor(x+y)
    a = 1
    b = 2
end

### [Polynomials.jl](https://github.com/JuliaMath/Polynomials.jl)

Approximate data by a polynomial to be able to calculate roots, extrema, integrals, etcetera.

In [ ]:
using Polynomials, Random

In [ ]:
myfunc(x) = 3*x
xdata = -1:.05:1
ydata = myfunc.(xdata) .+ 0.2*randn(length(xdata));
p = Polynomials.fit(xdata, ydata, 1)

In [ ]:
using PyPlot
grid = range(-1, 1, length=100)
plot(grid, p.(grid), label="polynomial")
plot(xdata, ydata, "o", label="data")
legend();

### [LsqFit.jl](https://github.com/JuliaNLSolvers/LsqFit.jl)

Least square fitting of data to a custom model.

In [ ]:
using LsqFit, Random

In [ ]:
model(x, p) = p[1]*exp.(-x.*p[2]) # f(x) = α*exp(-γ*x)

xdata = range(0, 10, length=20)
ydata = model(xdata, [1.0 2.0]) + 0.02*randn(length(xdata))
p0 = [0.5, 0.5]

fit = curve_fit(model, xdata, ydata, p0);

In [ ]:
fit.param

In [ ]:
# define function for fit result
modelfit = x -> model(x, fit.param)
modelfit(0.2)

In [ ]:
using PyPlot
grid = range(0, 10, length=100)
plot(grid, modelfit.(grid), label="polynomial")
plot(xdata, ydata, "o", label="data")
legend();

### [Measurements.jl](https://github.com/JuliaPhysics/Measurements.jl)

A package that allows you to define numbers with uncertainties, perform calculations involving them, and easily get the uncertainty of the result according to linear error propagation theory.

In [ ]:
using Measurements

In [ ]:
x = 4 ± 0.1

In [ ]:
typeof(x)

In [ ]:
y = measurement(5.1, 0.2)

In [ ]:
x + y

In [ ]:
x * y

In [ ]:
1/x # Δ(1/x) = d(1/x)/dx * Δx

Some properties to be aware of:

In [ ]:
(3 ± 0.1) === (3 ± 0.1)

In [ ]:
(3 ± 0.1) / (3 ± 0.1)

### [Unitful.jl](https://github.com/PainterQubits/Unitful.jl)

In [ ]:
using Unitful

In [ ]:
1u"kg"

In [ ]:
unit(1u"kg")

In [ ]:
ustrip(1u"kg") # strip the units

In [ ]:
uconvert(u"g", 1u"kg")

In [ ]:
1u"A" * 2u"Ω" isa Unitful.Voltage

Great, but can we avoid this `u"kg"` notation and make it more natural? Sure!

In [ ]:
using Unitful: ms, s, minute, hr, rad, °, mm, cm, m, km

In [ ]:
t = 1.0s

In [ ]:
t + 2s

In [ ]:
2*t

In [ ]:
t^2

In [ ]:
t + t^2

Note that in Julia using unitful quantities comes with only a minor overhead, in contrast to Python for example. For more information have a look at https://medium.com/@Jernfrost/defining-custom-units-in-julia-and-python-513c34a4c971.

Domain specific extensions available: [UnitfulAstro.jl](https://github.com/JuliaAstro/UnitfulAstro.jl), [UnitfulUS.jl](https://github.com/ajkeller34/UnitfulUS.jl), [UnitfulAngles.jl](https://github.com/yakir12/UnitfulAngles.jl)

### [PeriodicTable.jl](https://github.com/JuliaPhysics/PeriodicTable.jl)

In [ ]:
using PeriodicTable

In [ ]:
elements

In [ ]:
elements["Sodium"] # or elements[:Na] or elements[11]

### [HDF5.jl](https://github.com/JuliaIO/HDF5.jl)

In [ ]:
using HDF5

In [ ]:
A = rand(2,3)

In [ ]:
h5write("test.h5", "A", A)

In [ ]:
Aloaded = h5read("test.h5", "A")

If one wants to read/write multiple things at once one can do that like so:

In [ ]:
# r = read existing file
# r+ = read and write existing file
# w = overwrite existing / create new file
h5open("test.h5", "r+") do f
    for k in 1:5
        f[string(k)] = rand(2,2)
        # or write(f, string(k), rand(2,2))
    end
end

In [ ]:
f = h5open("test.h5", "r+")

In [ ]:
f["A"]

In [ ]:
read(f["A"])

In [ ]:
close(f);

HDF5 is nice and fine. It is supported by practically every programming language. However, some things are annoyingly complicated. For example, one can't just overwrite a dataset. Instead one has to check whether this dataset already exists, delete it if so, and then create a new dataset.

In [ ]:
h5open("test.h5", "r+") do f
    haskey(f, "A") && delete_object(f, "A")
    f["A"] = rand(2, 2)
end

In [ ]:
rm("test.h5");

### [JLD2.jl](https://github.com/JuliaIO/JLD2.jl)

HDF5 based but can story arbitrary Julia objects.

In [ ]:
using JLD2

In [ ]:
A = rand(2,3)

In [ ]:
@save "test.jld2" A

In [ ]:
B = rand(2,3)

In [ ]:
@save "test.jld2" B # overwrites(!) test.jld2 file

In [ ]:
@load "test.jld2" B # will load dataset "B" automatically to variable B

To get more control, one can open/close the file as for HDF5:

In [ ]:
jldopen("test.jld2", "r+") do f
    @show f["B"] # no read necessary
    @show f["C"] = rand(BigFloat, 2, 2); # can store arbitrary Julia objects
end;

In [ ]:
@load "test.jld2" # loads everything from file to variables

In [ ]:
C

In [ ]:
# clean up
rm("test.jld2")

### [PyCall.jl](https://github.com/JuliaPy/PyCall.jl)

In [ ]:
using PyCall

In [ ]:
np = pyimport("numpy")

In [ ]:
x = rand(2,2)

In [ ]:
np.linalg.eig(x)

### [Distributions.jl](https://github.com/JuliaStats/Distributions.jl)

In [ ]:
using Distributions

In [ ]:
d = Normal()

In [ ]:
rand(d, 10)

In [ ]:
d = Binomial()

In [ ]:
rand(d, 10)

### [StatsBase.jl](https://github.com/JuliaStats/StatsBase.jl)

In [ ]:
using StatsBase, Random

In [ ]:
x = randn(100);

In [ ]:
h = fit(Histogram, rand(100), nbins=10)

In [ ]:
countmap(rand(1:5, 100))

### [Revise.jl](https://github.com/timholy/Revise.jl)

Among others Revise can track

* any package that you load with `import` or `using`
* any script you load with `includet`

### [OMEinsum.jl](https://github.com/under-Peter/OMEinsum.jl)

In [ ]:
using OMEinsum

In [ ]:
x, y = rand(3), rand(3);

In [ ]:
ein"i->"(x)

In [ ]:
ein"i,i->"(x,y)

In [ ]:
ein"i,j->ij"(x,y)

In [ ]:
x*y'

In [ ]:
a, b = rand(2,2), rand(2,2);

In [ ]:
ein"ij,jk->ik"(a,b)

In [ ]:
a * b

In [ ]:
ein"ij,jk->"(a,b)

In [ ]:
ein"ij,jk->"(a,b)[] ≈ sum(a * b)

Note that there is also the more mature [Einsum.jl](https://github.com/ahwillia/Einsum.jl) and other options like [Tullio.jl](https://github.com/mcabbott/Tullio.jl).

### [TimerOutputs.jl](https://github.com/KristofferC/TimerOutputs.jl)

In [ ]:
using TimerOutputs

In [ ]:
const to = TimerOutput();

In [ ]:
function time_test()
    @timeit to "nest 1" begin
        sleep(0.1)
        # 3 calls to the same label
        @timeit to "level 2.1" sleep(0.03)
        @timeit to "level 2.1" sleep(0.03)
        @timeit to "level 2.1" sleep(0.03)
        @timeit to "level 2.2" sleep(0.2)
    end
    @timeit to "nest 2" begin
        @timeit to "level 2.1" sleep(0.3)
        @timeit to "level 2.2" sleep(0.4)
    end
end

time_test()

In [ ]:
to